### Week 1: Excercise
- To demonstrate your familiarity with OpenAI API, and also Ollama
- Build a tool that takes a technical question and responds with an explanation. 
- This is a tool that you will be able to use yourself during the course!

In [ ]:
# Import required libraries

In [ ]:
# Define constants for Model to be used
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming

In [ ]:
# Get Llama 3.2 to answer

### Example - Using Gpt-4o-mini

### Example - Using Llama3.2 (local docker env.)